In [1]:
from SmoothSparseWeights import *
from SmoothSparseWeights_Alternating import *

In [2]:
# exampleName="cheburashka"
# poses_folder="poses-2"
# poses_ind=7
# clip_k=4
# handle_num=11


exampleName="cow"
poses_folder="poses-2"
poses_ind=5
clip_k=4
handle_num=15


# exampleName="wolf"
# poses_folder="poses-1"
# poses_ind=2
# clip_k=4
# handle_num=21


objective_choice=0


weights = {
           'W_sparse': 0.0,
           'W_spatial': 0.0,
           'W_sum': 10.0
          }

In [3]:
#### read obj file

### rest_poses:
vertices_obj_path="./models/"+exampleName+"-all/"+exampleName+".obj"
mesh0=TriMesh.FromOBJ_FileName(vertices_obj_path)

### other poses
vertices_obj_dir="./models/"+exampleName+"-all/"+poses_folder+"/"
filenames=glob.glob(vertices_obj_dir+"*.obj")
print(filenames)

mesh1_list=[]
for i in range(len(filenames)):
    mesh1_list.append(TriMesh.FromOBJ_FileName(filenames[i]))


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 


['./models/cheburashka-all/poses-2/cheburashka-01.obj', './models/cheburashka-all/poses-2/cheburashka-02.obj', './models/cheburashka-all/poses-2/cheburashka-03.obj', './models/cheburashka-all/poses-2/cheburashka-04.obj', './models/cheburashka-all/poses-2/cheburashka-05.obj', './models/cheburashka-all/poses-2/cheburashka-06.obj', './models/cheburashka-all/poses-2/cheburashka-07.obj', './models/cheburashka-all/poses-2/cheburashka-08.obj', './models/cheburashka-all/poses-2/cheburashka-09.obj', './models/cheburashka-all/poses-2/cheburashka-10.obj', './models/cheburashka-all/poses-2/cheburashka-11.obj']
(6669, 44)
(6669, 33)


In [4]:
#### solve for transformation per vertex, with smoothness regularization and transformation matrix L2 norm regularization.
output_prefix="./sparse-smooth-mixing-weights-test/"+exampleName


In [5]:
#### read gt per vertex mixing weights 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+".DMAT")
print(groundtruth_names)
gt=load_DMAT(groundtruth_names[0])
gt_mixing_weights=gt.T.copy()
print (gt_mixing_weights.shape)

print (((gt_mixing_weights.sum(axis=-1)-1)**2).sum())

['./models/cheburashka-all/cheburashka.DMAT']
(6669, 11)
7.810955556852425e-29


In [6]:
### load groundtruth endmember
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.Tmat")
print(groundtruth_names)

gt_handles=None
for i in range(len(groundtruth_names)):
    gt=load_Tmat(groundtruth_names[i])
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    print (gt.shape)
    if gt_handles is None:
        gt_handles=gt
    else:
        gt_handles=np.hstack((gt_handles, gt))
    
print(gt_handles.shape)
gt_handles=gt_handles.reshape(-1, len(groundtruth_names), 12)


# #### is this True??????
# gt_handles=np.transpose(gt_handles.reshape(-1, len(groundtruth_names), 12), (1,0,2)) 


print(gt_handles.shape)

for i in range(len(gt_handles)):
    print (gt_handles[i][2].reshape((3,4)))

test=gt_handles[1][2].reshape((3,4))[:,:3]
print (test.T.dot(test))

['./models/cheburashka-all/poses-2/cheburashka-01.Tmat', './models/cheburashka-all/poses-2/cheburashka-02.Tmat', './models/cheburashka-all/poses-2/cheburashka-03.Tmat', './models/cheburashka-all/poses-2/cheburashka-04.Tmat', './models/cheburashka-all/poses-2/cheburashka-05.Tmat', './models/cheburashka-all/poses-2/cheburashka-06.Tmat', './models/cheburashka-all/poses-2/cheburashka-07.Tmat', './models/cheburashka-all/poses-2/cheburashka-08.Tmat', './models/cheburashka-all/poses-2/cheburashka-09.Tmat', './models/cheburashka-all/poses-2/cheburashka-10.Tmat', './models/cheburashka-all/poses-2/cheburashka-11.Tmat']
(11, 12)
(11, 12)
(11, 12)
(11, 12)
(11, 12)
(11, 12)
(11, 12)
(11, 12)
(11, 12)
(11, 12)
(11, 12)
(11, 132)
(11, 11, 12)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.943272  -0.314168   0.107404   0.106593 ]
 [-0.0357635  0.225464   0.973595  -0.0999588]
 [-0.330088  -0.922206   0.201439   0.882928 ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0

In [7]:
#### do a groundtruth test.
val=objective_func_vector(gt_mixing_weights.ravel(), vertices1, vertices2, gt_handles, gt_mixing_weights.ravel(), gt_mixing_weights.ravel(), choice=objective_choice)
print ((abs(val)).max(), (abs(val)).min())
print (np.square(val).sum()/(len(val)))

res=run(mesh0, mesh1_list, output_prefix, weights, gt_handles, gt_mixing_weights.ravel(), gt_mixing_weights.ravel(), initials=gt_mixing_weights.ravel(), choice=objective_choice)
res=res.reshape(gt_mixing_weights.shape)

objective function choice:  0
0.00016590000000005212 0.0
6.309560751784238e-13
(6669, 44)
(6669, 33)
objective function choice:  0
1.8928682255352715e-12
objective function choice:  0
Autograd ArrayBox with value 1.8928682255352715e-12
True
1.2623538196094724e-08


In [8]:
#### read gt per vertex transformation 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.DMAT")
print(groundtruth_names)

gt_all=None
for i in range(len(groundtruth_names)):
    gt=load_DMAT(groundtruth_names[i])
    gt=gt.T
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    print (gt.shape)
    if gt_all is None:
        gt_all=gt
    else:
        gt_all=np.hstack((gt_all, gt))
    
print(gt_all.shape)


['./models/cheburashka-all/poses-2/cheburashka-01.DMAT', './models/cheburashka-all/poses-2/cheburashka-02.DMAT', './models/cheburashka-all/poses-2/cheburashka-03.DMAT', './models/cheburashka-all/poses-2/cheburashka-04.DMAT', './models/cheburashka-all/poses-2/cheburashka-05.DMAT', './models/cheburashka-all/poses-2/cheburashka-06.DMAT', './models/cheburashka-all/poses-2/cheburashka-07.DMAT', './models/cheburashka-all/poses-2/cheburashka-08.DMAT', './models/cheburashka-all/poses-2/cheburashka-09.DMAT', './models/cheburashka-all/poses-2/cheburashka-10.DMAT', './models/cheburashka-all/poses-2/cheburashka-11.DMAT']
(6669, 12)
(6669, 12)
(6669, 12)
(6669, 12)
(6669, 12)
(6669, 12)
(6669, 12)
(6669, 12)
(6669, 12)
(6669, 12)
(6669, 12)
(6669, 132)


In [9]:
print (gt_mixing_weights.shape)
print (gt_handles.shape)

(6669, 11)
(11, 11, 12)


In [10]:
per_vertex_trans=gt_mixing_weights.dot(gt_handles.reshape((gt_mixing_weights.shape[1], -1)))
print (per_vertex_trans.shape)

(6669, 132)


In [11]:
diff= abs(gt_all-per_vertex_trans.reshape((len(gt_all),-1)))

print (diff.max(),diff.min())
print (diff.sum())

0.0007782307929949939 0.0
0.212554458075536


In [12]:
Matrix=gt_all.copy()

N=len(vertices1)
Matrix=Matrix.reshape((N, -1, 3, 4))

reconstruct_vertices2=np.multiply(Matrix, vertices1.reshape((N, -1, 1, 4))).sum(axis=-1)

err= (reconstruct_vertices2-vertices2.reshape((N,-1,3))).reshape(-1)

In [13]:
print (np.square(err).sum()/len(err))

1.585319064374342e-15
